# Preliminaries

In [1]:
import numpy as np
import matplotlib.pylab as plt
import pint

from dissipationtheory.constants import ureg, qe, epsilon0

In [2]:
from dissipationtheory.dissipation7 import CantileverModel as CantileverModel_v7
from dissipationtheory.dissipation7 import SampleModel1 as SampleModel1_v7
from dissipationtheory.dissipation7 import SampleModel2 as SampleModel2_v7
from dissipationtheory.dissipation7 import theta1norm, theta2norm
from dissipationtheory.dissipation7 import K as K_v7

In [3]:
from dissipationtheory.dissipation8a import CantileverModel as CantileverModel_v8a
from dissipationtheory.dissipation8a import SampleModel1 as SampleModel1_v8a
from dissipationtheory.dissipation8a import SampleModel2 as SampleModel2_v8a
from dissipationtheory.dissipation8a import integrand1, integrand2
from dissipationtheory.dissipation8a import K as K_v8a

In [4]:
from dissipationtheory.dissipation8b import CantileverModelJit, SampleModel1Jit, SampleModel2Jit
from dissipationtheory.dissipation8b import integrand1jit, integrand2jit
from dissipationtheory.dissipation8b import K_jit 

# (Study A) Version 7 sample Python objects

Instantiate representative cantilever, sample1, and sample2 objects.

In [5]:
cantilever_v7 = CantileverModel_v7(
    f_c = ureg.Quantity(75, 'kHz'),
    k_c = ureg.Quantity(2.8, 'N/m'), 
    V_ts = ureg.Quantity(1, 'V'), 
    R = ureg.Quantity(35, 'nm'),
    angle = ureg.Quantity(20, 'degrees'),
    d = ureg.Quantity(38, 'nm'),
    z_c = ureg.Quantity(20, 'nm')
)

In [6]:
sample1_v7 = SampleModel1_v7(
    cantilever = cantilever_v7,
    h_s = ureg.Quantity(100, 'nm'),
    epsilon_s = ureg.Quantity(complex(20, -0.2), ''),
    sigma = ureg.Quantity(1E-5, 'S/m'),
    rho = ureg.Quantity(1e21, '1/m^3'),
    epsilon_d = ureg.Quantity(complex(1e6, 0), ''),
    z_r = ureg.Quantity(300, 'nm')
)

In [7]:
sample1_v7

cantilever

         resonance freq = 75.000 kHz
                        = 4.712e+05 rad/s
        spring constant = 2.800 N/m
     tip-sample voltage = 1.000 V
                 radius = 35.000 nm
        cone half angle = 20.000 degree
                 height = 38.000 nm
  tip charge z location = 20.000 nm

semiconductor

             epsilon (real) = 20.000
             epsilon (imag) = -0.200
                  thickness = 100.0 nm
               conductivity = 1.000e-05 S/m
             charge density = 1.000e+21 m^{-3}
           reference height = 3.000e+02 nm

         roll-off frequency = 1.129e+06 Hz
                   mobility = 6.242e-08 m^2/(V s)
         diffusion constant = 1.614e-09 m^2/s
               Debye length = 3.780e+01 nm
           diffusion length = 5.852e+01 nm
   effective epsilon (real) = 20.000
   effective epsilon (imag) = -2.597

dielectric

  epsilon (real) = 1000000.000
  epsilon (imag) = 0.000
       thickness = infinite

In [8]:
sample2_v7 = SampleModel2_v7(
    cantilever = cantilever_v7,
    epsilon_d = ureg.Quantity(complex(20, -0.2), ''),
    h_d = ureg.Quantity(50, 'nm'),
    epsilon_s = ureg.Quantity(complex(20, -0.2), ''),
    sigma = ureg.Quantity(1E-5, 'S/m'),
    rho = ureg.Quantity(1e21, '1/m^3'),
    z_r = ureg.Quantity(300, 'nm')
)       

In [9]:
sample2_v7

cantilever

         resonance freq = 75.000 kHz
                        = 4.712e+05 rad/s
        spring constant = 2.800 N/m
     tip-sample voltage = 1.000 V
                 radius = 35.000 nm
        cone half angle = 20.000 degree
                 height = 38.000 nm
  tip charge z location = 20.000 nm

dielectric

  epsilon (real) = 20.000
  epsilon (imag) = -0.200
       thickness = 50.0 nm

semiconductor

             epsilon (real) = 20.000
             epsilon (imag) = -0.200
                  thickness = infinite
               conductivity = 1.000e-05 S/m
             charge density = 1.00e+21 m^{-3}
           reference height = 300.0 nm

         roll-off frequency = 1.129e+06 Hz
                   mobility = 6.24e-08 m^2/(V s)
         diffusion constant = 1.61e-09 m^2/s
               Debye length = 37.8 nm
           diffusion length = 58.5 nm
   effective epsilon (real) = 20.000
   effective epsilon (imag) = -2.597


Confirm that the `theta1norm` function runs.

In [10]:
theta1norm(1.01, sample1_v7, ureg.Quantity(1e3,'Hz'), 2, part=np.real)

0.8658450727436651

Confirm that the `theta2norm` function runs.

In [11]:
theta2norm(1.01, sample2_v7, ureg.Quantity(1e3,'Hz'), 2, part=np.real)

0.8387173454896705

Compute $K_2$ for Sample 1.

In [12]:
resultA1 = K_v7(2, theta1norm, sample1_v7, ureg.Quantity(0.001,'Hz'), part=np.real)
resultA1.to('1/um**3')

<Quantity(28320.292826217443, '1 / micrometer ** 3')>

Compute $K_2$ for Sample 2.

In [13]:
resultA2 = K_v7(2, theta2norm, sample2_v7, ureg.Quantity(0.001,'Hz'), part=np.real)
resultA2.to('1/um**3')

<Quantity(28279.857978576692, '1 / micrometer ** 3')>

# (Study B) Version 8a sample Python objects

I had added cantilever length $L$ to the list of cantilever parameters.  

In [14]:
cantilever_v8a = CantileverModel_v8a(
    f_c = ureg.Quantity(75, 'kHz'),
    k_c = ureg.Quantity(2.8, 'N/m'), 
    V_ts = ureg.Quantity(1, 'V'), 
    R = ureg.Quantity(35, 'nm'),
    angle = ureg.Quantity(20, 'degree'),
    L = ureg.Quantity(1000, 'nm'), 
    d = ureg.Quantity(38, 'nm')
)

Make sure we can generate a Model I sample object ok.

In [15]:
sample1_v8a = SampleModel1_v8a(
    cantilever = cantilever_v8a,
    h_s = ureg.Quantity(100, 'nm'),
    epsilon_s = ureg.Quantity(complex(20, -0.2), ''),
    sigma = ureg.Quantity(1E-5, 'S/m'),
    rho = ureg.Quantity(1e21, '1/m^3'),
    epsilon_d = ureg.Quantity(complex(1e6, 0), ''),
    z_r = ureg.Quantity(300, 'nm')
)

In [16]:
sample1_v8a

cantilever

         resonance freq = 75.000 kHz
                        = 4.712e+05 rad/s
        spring constant = 2.800 N/m
     tip-sample voltage = 1.000 V
                 radius = 35.000 nm
        cone half angle = 20.000 degree
            cone length = 1000.000 nm
  tip-sample separation = 38.000 nm

semiconductor

             epsilon (real) = 20.000
             epsilon (imag) = -0.200
                  thickness = 100.0 nm
               conductivity = 1.000e-05 S/m
             charge density = 1.000e+21 m^{-3}
           reference height = 3.000e+02 nm

         roll-off frequency = 1.129e+06 Hz
       inverse Debye length = 2.646e-02 nm^{-1}
               Debye length = 3.780e+01 nm
dielectric

  epsilon (real) = 1000000.000
  epsilon (imag) = 0.000
       thickness = infinite

Make sure we can generate a Model II sample object ok.

In [17]:
sample2_v8a = SampleModel2_v8a(
    cantilever = cantilever_v8a,
    epsilon_d = ureg.Quantity(complex(20, -0.2), ''),
    h_d = ureg.Quantity(50, 'nm'),
    epsilon_s = ureg.Quantity(complex(20, -0.2), ''),
    sigma = ureg.Quantity(1E-5, 'S/m'),
    rho = ureg.Quantity(1e21, '1/m^3'),
    z_r = ureg.Quantity(300, 'nm')
)       

In [18]:
sample2_v8a

cantilever

         resonance freq = 75.000 kHz
                        = 4.712e+05 rad/s
        spring constant = 2.800 N/m
     tip-sample voltage = 1.000 V
                 radius = 35.000 nm
        cone half angle = 20.000 degree
            cone length = 1000.000 nm
  tip-sample separation = 38.000 nm

dielectric

  epsilon (real) = 20.000
  epsilon (imag) = -0.200
       thickness = 50.0 nm

semiconductor

             epsilon (real) = 20.000
             epsilon (imag) = -0.200
                  thickness = infinite
               conductivity = 1.000e-05 S/m
             charge density = 1.00e+21 m^{-3}
           reference height = 300.0 nm

         roll-off frequency = 1.129e+06 Hz
       inverse Debye length = 2.646e-02 nm^{-1}
               Debye length = 3.780e+01 nm

Check that the new `integrand1` function runs.  Check the result at $\omega = 0$.  Note that the prior function `theta2norm` would blow up at $\omega = 0$, requiring us to call the function as a small but finite $\omega$.  So being able to compute the integrand at $\omega = 0$ exactly is progress.

Check that the new `integrand2` function runs.

In [19]:
integrand1(1.01, 2, sample1_v8a, ureg.Quantity(0,'Hz'), ureg.Quantity([2, 5, 10], 'nm'), ureg.Quantity([2, 5, 10], 'nm'), part=np.real)

0.9261562258439279

In [20]:
integrand2(1.01, 2, sample2_v8a, ureg.Quantity(0,'Hz'), ureg.Quantity([2, 5, 10], 'nm'), ureg.Quantity([2, 5, 10], 'nm'), part=np.real)

0.897138242296477

Compute $K_2$ for Sample 1.

In [21]:
resultB1 = K_v8a(integrand1, 2, sample1_v8a, ureg.Quantity(0.001,'Hz'), ureg.Quantity([0, 0, 20], 'nm'), ureg.Quantity([0, 0, 20], 'nm'), part=np.real)
resultB1.to('1/um**3')

<Quantity(28320.29282621744, '1 / micrometer ** 3')>

Compute $K_2$ for Sample 2.

In [22]:
resultB2 = K_v8a(integrand2, 2, sample2_v8a, ureg.Quantity(0.001,'Hz'), ureg.Quantity([0, 0, 20], 'nm'), ureg.Quantity([0, 0, 20], 'nm'), part=np.real)
resultB2.to('1/um**3')

<Quantity(28279.857978576692, '1 / micrometer ** 3')>

# (Study C) Version 8b sample jit objects

This works

In [23]:
cantilever_jit = CantileverModelJit(
    f_c = 75.0e3,
    k_c = 2.8,
    V_ts = 1.0,
    R = 35.0e-9,
    angle = 20.0,
    L = 1000.0e-9,
    d = 38.0e-9
)

These also work, yea!

In [24]:
cantilever_jit = CantileverModelJit(**cantilever_v8a.args())

In [25]:
sample1_jit = SampleModel1Jit(**sample1_v8a.args())
sample1_jit.print()

cantilever
        cantilever freq =  75000.0 Hz
                        =  471238.89803846896 rad/s
        spring constant =  2.8 N/m
     tip-sample voltage =  1.0 V
                 radius =  3.5e-08 m
        cone half angle =  20.0 degree
            cone length =  1.0000000000000002e-06 m
  tip-sample separation =  3.8e-08 m

semiconductor
          epsilon (real) =  20.0
          epsilon (imag) =  -0.2
               thickness =  1.0000000000000001e-07 m
            conductivity =  1e-05 S/m
          charge density =  1e+21 m^{{-3}}
        reference height =  3.0000000000000004e-07 m
 
      roll-off frequency =  1129409.0673730192 Hz
    inverse Debye length =  26456583.416667342 m^{{-1}}
            Debye length =  3.7797775481848936e-08 m

dielectric
 epsilon (real) =  1000000.0
 epsilon (imag) =  0.0
      thickness = infinite


In [26]:
sample2_jit = SampleModel2Jit(**sample2_v8a.args())
sample2_jit.print()

cantilever
        cantilever freq =  75000.0 Hz
                        =  471238.89803846896 rad/s
        spring constant =  2.8 N/m
     tip-sample voltage =  1.0 V
                 radius =  3.5e-08 m
        cone half angle =  20.0 degree
            cone length =  1.0000000000000002e-06 m
  tip-sample separation =  3.8e-08 m

dielectric
 epsilon (real) =  20.0
 epsilon (imag) =  -0.2
      thickness =  5.0000000000000004e-08 m

semiconductor
          epsilon (real) =  20.0
          epsilon (imag) =  -0.2
               thickness = infinite
            conductivity =  1e-05 S/m
          charge density =  1e+21 m^{{-3}}
        reference height =  3.0000000000000004e-07 m
 
      roll-off frequency =  1129409.0673730192 Hz
    inverse Debye length =  26456583.416667342 m^{{-1}}
            Debye length =  3.7797775481848936e-08 m


Check the integrand functions.  Compare the execution time in jit versus pure Python.

In [27]:
%%time
integrand2jit(1.01, 2, sample2_jit, 0., np.array([2e-9, 5e-9, 10e-9]), np.array([2e-9, 5e-9, 10e-9]), False)

CPU times: user 139 μs, sys: 101 μs, total: 240 μs
Wall time: 246 μs


0.8971382422964769

In [28]:
%%time
integrand2(1.01, 2, sample2_v8a, ureg.Quantity(0,'Hz'), ureg.Quantity([2, 5, 10], 'nm'), ureg.Quantity([2, 5, 10], 'nm'), part=np.real)

CPU times: user 4.42 ms, sys: 949 μs, total: 5.36 ms
Wall time: 5.01 ms


0.897138242296477

Pleasingly, we get nearly the same result using both jit and Python. The jit function is about 100-fold faster.

Compute $K_2$ for Sample 1 using jit code.

In [29]:
%%time
resultC1 = K_jit(integrand1jit, 2, sample1_jit, 0.001, np.array([0, 0, 20e-9]), np.array([0, 0, 20e-9]), False)
resultC1

CPU times: user 2.28 ms, sys: 329 μs, total: 2.61 ms
Wall time: 3.46 ms


<Quantity(2.832029282621744e+22, '1 / meter ** 3')>

Compute $K_2$ for Sample 2 using jit code.

In [30]:
%%time
resultC2 = K_jit(integrand2jit, 2, sample2_jit, 0.001, np.array([0, 0, 20e-9]), np.array([0, 0, 20e-9]), False)
resultC2

CPU times: user 1.43 ms, sys: 61 μs, total: 1.49 ms
Wall time: 1.54 ms


<Quantity(2.82798579785767e+22, '1 / meter ** 3')>

We get similar results, but the jit code is 500- to 1000-fold faster.

# Compare results


Compare $K_2$ computed various ways. The various ways are 

* (Study A) "old pure Python" `v7`
* (Study B) "new pure Python" `v8a`
* (Study C) "new jit" `v8b`

Display the fractional error, taking the `v7` code as the ground truth.

In [31]:
def compare(A,B):
    """Compute the relative difference between A to B, taking A as the ground truth."""
    print("relative error =", ((A - B)/A).to('dimensionless').magnitude)

## Compare Study A to Study B

Sample 1

In [32]:
compare(resultA1, resultB1)

relative error = 1.4810242343670728e-16


Sample 2

In [33]:
compare(resultA2, resultB2)

relative error = 0.0


## Compare Study A to Study C

Sample 1

In [34]:
compare(resultA1, resultC1)

relative error = 1.4810242343670728e-16


Sample 2

In [35]:
compare(resultA2, resultC2)

relative error = -1.4831418188794937e-16


# Conclusions

Finally, we emphasize that the new `K` (in `dissipation8a.py`) and `K_jit` (in `dissipation8b.py`) functions are well behaved at zero frequency.  Here are two examples, computing $K_0$ for Sample 1 and Sample 2 at $\omega = 0$.

In [37]:
K_jit(
    integrand=integrand1jit, 
    power=0, 
    sample=sample1_jit, 
    omega=0,
    location1=np.array([0, 0, 20e-9]),
    location2=np.array([0, 0, 20e-9]), 
    isImag=False).to('1/um')

<Quantity(23.206670538554317, '1 / micrometer')>

In [39]:
K_jit(
    integrand=integrand2jit, 
    power=0, 
    sample=sample2_jit, 
    omega=0,
    location1=np.array([0, 0, 20e-9]),
    location2=np.array([0, 0, 20e-9]), 
    isImag=False).to('1/um')

<Quantity(22.910064305128017, '1 / micrometer')>

<div class="alert alert-block alert-success"> We have coded the functions $\theta_{I}$, $\theta_{II}$, $r_{\mathrm{p}}$, and $K_n$, in both pure Python and jit, in such a way that these functions are well-behaved at $\omega = 0$. For representative type I and type II samples, at finite $\omega = 0.001 \: \mathrm{Hz}$ we obtain the same $K_2$ values as obtained with prior Python code. </div>

# Explore vectorization

We want to loop over the frequency (to compute a BDLS spectrum) or over locations (to compute the response function at various heights).  It would be nice to not have to write loops all the time. Look into the `np.vectorize` function.  

In the following example, I am able to loop over a *list* of frequencies $\omega$.  This looping would be useful in computing a BLDS spectrum.  In the function below, the `otypes=[object]` is needed because the `K_jit` function returns a `pint` object with units.

In [41]:
# the arguments: integrand, power, sample, omega, location, isImag

K_jit_vectorized = np.vectorize(K_jit, otypes=[object], excluded=['integrand','power','sample','location1','location2','isImag'])

K_jit_vectorized(
    integrand=integrand2jit, 
    power=2, 
    sample=sample2_jit, 
    omega=[0.000, 0.001],
    location1=np.array([0, 0, 20e-9]), 
    location2=np.array([0, 0, 20e-9]), 
    isImag=False)

array([<Quantity(2.827985797857637e+22, '1 / meter ** 3')>,
       <Quantity(2.82798579785767e+22, '1 / meter ** 3')>], dtype=object)

This same approach does not enable looping over the vector `location`.  The following code fails.  I tried using the `signature` argument, but this approach failed.

In [43]:
if 0:
    K_jit_vectorized = np.vectorize(K_jit, otypes=[object], excluded=['integrand','power','sample','omega','isImag'])
    
    K_jit_vectorized(
        integrand=integrand2jit, 
        power=2, 
        sample=sample2_jit, 
        omega=0.001,
        location1=[np.array([0, 0, 20e-9]),np.array([0, 0, 40e-9])],
        location2=[np.array([0, 0, 20e-9]),np.array([0, 0, 40e-9])],
        isImag=False)

Idea: We could just use a list comprehension ...

In [44]:
result = [K_jit(integrand2jit, 2, sample2_jit, 0.001, location, location, False)
     for location in [np.array([0, 0, 20e-9]), np.array([0, 0, 40e-9])]]

... and turn the resulting list into a numpy array with units using the `pint.Quantity.from_list` utility function.

In [55]:
pint.Quantity.from_list(result).to('1/um^3')

Magnitude,[28279.857978576696 3538.1363511929026]
Units,1/micrometer3


Other ideas

* `np.outer`
* `np.nditer`

# Check that $K_0$ over a "metal" is the expected image potential

## Sample 1

In [46]:
sample1_metal = SampleModel1_v8a(
    cantilever = cantilever_v8a,
    h_s = ureg.Quantity(1e-6, 'nm'),  # not zero
    epsilon_s = ureg.Quantity(complex(1, 0), ''),
    sigma = ureg.Quantity(1e9, 'S/m'),
    rho = ureg.Quantity(1e26, '1/m^3'),
    epsilon_d = ureg.Quantity(complex(1e6, 0), ''),
    z_r = ureg.Quantity(30, 'nm')
)

In [47]:
sample1_jit_metal = SampleModel1Jit(**sample1_metal.args())
sample1_jit_metal.print()

cantilever
        cantilever freq =  75000.0 Hz
                        =  471238.89803846896 rad/s
        spring constant =  2.8 N/m
     tip-sample voltage =  1.0 V
                 radius =  3.5e-08 m
        cone half angle =  20.0 degree
            cone length =  1.0000000000000002e-06 m
  tip-sample separation =  3.8e-08 m

semiconductor
          epsilon (real) =  1.0
          epsilon (imag) =  0.0
               thickness =  1e-15 m
            conductivity =  1000000000.0 S/m
          charge density =  1e+26 m^{{-3}}
        reference height =  3.0000000000000004e-08 m
 
      roll-off frequency =  1.129409067373019e+20 Hz
    inverse Debye length =  8366306270.290836 m^{{-1}}
            Debye length =  1.1952706101031096e-10 m

dielectric
 epsilon (real) =  1000000.0
 epsilon (imag) =  0.0
      thickness = infinite


Compute $K_0$ for this "metallic" type 1 sample.

In [48]:
K_I = K_jit(
    integrand=integrand1jit, 
    power=0, 
    sample=sample1_jit_metal, 
    omega=0,
    location1=np.array([  0,     0, 10e-9]),
    location2=np.array([3e-9, 5e-9, 10e-9]), 
    isImag=False).to('1/um')

K_I

<Quantity(48.00143564666106, '1 / micrometer')>

Compute the expected unitless image potential.  We see that it agrees closely $K_0$ compute above.

In [49]:
K_image = (1/ureg.Quantity(np.sqrt(3**2 + 5**2 + 20**2),'nm')).to('1/um')
K_image

<Quantity(48.00153607373192, '1 / micrometer')>

Compare $K_0$ for type 1 metallic sample to the image potential.

In [50]:
compare(K_I, K_image)

relative error = -2.0921680676645053e-06


## Sample 2

In [51]:
sample2_metal = SampleModel2_v8a(
    cantilever = cantilever_v8a,
    epsilon_d = ureg.Quantity(complex(1, 0), ''),
    h_d = ureg.Quantity(0, 'nm'), # can be zero
    epsilon_s = ureg.Quantity(complex(1e6, 0), ''),
    sigma = ureg.Quantity(1e9, 'S/m'),
    rho = ureg.Quantity(1e26, '1/m^3'),
    z_r = ureg.Quantity(50, 'nm')
)

In [52]:
sample2_jit_metal = SampleModel2Jit(**sample2_metal.args())
sample2_jit_metal.print()

cantilever
        cantilever freq =  75000.0 Hz
                        =  471238.89803846896 rad/s
        spring constant =  2.8 N/m
     tip-sample voltage =  1.0 V
                 radius =  3.5e-08 m
        cone half angle =  20.0 degree
            cone length =  1.0000000000000002e-06 m
  tip-sample separation =  3.8e-08 m

dielectric
 epsilon (real) =  1.0
 epsilon (imag) =  0.0
      thickness =  0.0 m

semiconductor
          epsilon (real) =  1000000.0
          epsilon (imag) =  0.0
               thickness = infinite
            conductivity =  1000000000.0 S/m
          charge density =  1e+26 m^{{-3}}
        reference height =  5.0000000000000004e-08 m
 
      roll-off frequency =  1.129409067373019e+20 Hz
    inverse Debye length =  8366306270.290836 m^{{-1}}
            Debye length =  1.1952706101031096e-10 m


Compute $K_0$ for this "metallic" type 2 sample.

In [53]:
K_II = K_jit(
    integrand=integrand2jit, 
    power=0, 
    sample=sample2_jit_metal, 
    omega=0,
    location1=np.array([  0,     0, 10e-9]),
    location2=np.array([3e-9, 5e-9, 10e-9]), 
    isImag=False).to('1/um')

K_II

<Quantity(48.00145327358575, '1 / micrometer')>

Compare $K_0$ for type 2 metallic sample to the image potential.

In [54]:
compare(K_II, K_image)

relative error = -1.7249508196157093e-06


### Conclusion

<div class="alert alert-block alert-success"> Over a metallic Type 1 and Type 2 sample, $K_0$ agrees with the expected image potential to within a few parts per million. </div>

::: {.content-hidden when-format="html"}

# Formatting notes

The header at the top of this file is for creating a nicely-formatted `.html` document using the program `quarto` ([link](https://quarto.org/)).  To create nicely-formated `.html`versions of this notebook, run `quarto` from the command line as follows

    quarto render dissipation-theory--Study-44.ipynb && open dissipation-theory--Study-44.html

Other useful information about this notebook:

- Filename: `dissipation-theory--Study-44.ipynb`
- Continued from: `dissipation-theory--Study-43.ipynb`
- Continued to: ---

:::